[refernece](https://cloud.google.com/vertex-ai/generative-ai/docs/start/quickstarts/quickstart-multimodal?hl=zh-cn#python)  
[colab](https://console.cloud.google.com/vertex-ai/colab/notebooks?project=linen-flash-428508-h4&hl=en&supportedpurview=project&activeNb=projects%2Flinen-flash-428508-h4%2Flocations%2Fus-central1%2Frepositories%2F930bb123-178f-4a38-b77a-c1c5e7341480)

In [1]:
!pip install -U -q google-cloud-aiplatform[evaluation]
!pip install -U -q datasets
!pip install -U -q anthropic[vertex]
!pip install -U -q openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.3 requires dill>=0.3.9, but you have dill 0.3.8 which is incompatible.
pathos 0.3.3 requires multiprocess>=0.70.17, but you have multiprocess 0.70.16 which is incompatible.


In [2]:
import IPython
from IPython.display import HTML, Markdown, display

app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [3]:
PROJECT_ID = "mimetic-kit-445917-d8"
LOCATION = "us-central1"  # @param {type:"string"}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

from anthropic import AnthropicVertex
from google.auth import default, transport
import openai
from vertexai.evaluation import (
    EvalTask,
    MetricPromptTemplateExamples,
    PairwiseMetric,
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
)
from vertexai.generative_models import GenerativeModel
import vertexai

from vertexai.generative_models import GenerativeModel, Part, Image

In [5]:
import logging
import warnings

import pandas as pd

logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

# pd.set_option('display.max_colwidth', None)

In [6]:
model = GenerativeModel(
    "gemini-1.5-pro-002",
    generation_config={"temperature": 0.6, "max_output_tokens": 256, "top_k": 1},
)

In [7]:
vertexai.init(project=PROJECT_ID, location="us-central1")

# Read the local image
local_image_path = "/home/lds/github/Causality-informed-Generation/20241216_004722.png"
system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships, highlighting key features or patterns that support your conclusions."
scene_info = "There are 3 variables: 1. the weight of cube, 2. the spring constant, and 3. the lenght of spring.You try to fill this causal adjacency matrix."
matrix = "[[_, _, _], [_ , _, _], [_ , _, _]]"
matrix_info = "in the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."


response = model.generate_content(
    [
        Part.from_image(Image.load_from_file(local_image_path)),
        Part.from_image(Image.load_from_file(local_image_path)),
        Part.from_image(Image.load_from_file(local_image_path)),
        Part.from_image(Image.load_from_file(local_image_path)),
        Part.from_image(Image.load_from_file(local_image_path)),
        Part.from_text(system_info + scene_info + matrix + matrix_info),
    ]
)


display(HTML("<h2>Model Response: </h2><hr>"))
Markdown(response.text)


The causal relationships are as follows:

1. **Weight of the cube causes a change in the length of the spring.**  A heavier cube will compress the spring more, resulting in a shorter length. A lighter cube will compress the spring less, resulting in a longer length.

2. **The spring constant influences the length of the spring.** A higher spring constant means the spring is stiffer and will compress less for a given weight, resulting in a longer length. A lower spring constant means the spring is less stiff and will compress more for a given weight, resulting in a shorter length.

3. **Neither the spring length nor the spring constant affect the weight of the cube.** The cube's weight is an independent variable.


Therefore, the causal adjacency matrix is:

```
[[0, 0, 1],
 [0, 0, 1],
 [0, 0, 0]]
```

Where:

* Row 1: Weight of the cube
* Row 2: Spring constant
* Row 3: Length of the spring


This matrix representation indicates that the weight (row 1) and spring constant (row 2) both cause a change in the spring length (column 3).

# inference

In [10]:
def extract_matrix(text):
  first_index = text.find("```")
  final_index = text.rfind("```")
  # print(text[first_index+3:final_index])
  if first_index == -1 or final_index == -1:
    raise ValueError("Matrix not found in the response")
    return None
  matrix = eval(text[first_index+3:final_index])
  return matrix
 
import logging
import csv
from datetime import datetime

def Gemini_infer(setting, csv_file="gemini_inference_log.csv"):
    # Set up logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger("Gemini_infer")

    # Initialize model with flexible configuration
    model = GenerativeModel(
        setting.get("model_name", "default-model-name"),
        generation_config=setting.get("generation_config", {"temperature": 1, "max_output_tokens": 1000, "top_k": 1}),
    )

    # Load images
    imgs = [Part.from_image(Image.load_from_file(path)) for path in setting["image_path"]]
    # logger.info("Loaded images: %s", setting["image_path"])

    # Build the prompt
    prompt = imgs + [Part.from_text(
        setting["system_info"] +
        setting["scene_info"] +
        setting["matrix"] +
        setting["matrix_info"]
    )]
    text_prompt = setting["system_info"] + setting["scene_info"] + setting["matrix"] + setting["matrix_info"]
    # Log the prompt for debugging
    # logger.info("Generated Prompt: %s", prompt)

    # Generate response
    try:
        response = model.generate_content(prompt)
    except Exception as e:
        logger.error("Model failed to generate content: %s", str(e))
        raise

    # Log the response for debugging
    # logger.info("Model Response: %s", response.text)

    # Extract matrix
    try:
        matrix = extract_matrix(response.text)
    except ValueError as e:
        logger.error("Matrix extraction failed: %s", str(e))
        raise

    # Log settings, prompt, response, and matrix
    # logger.info("Settings: %s", setting)
    # logger.info("Extracted Matrix: %s", matrix)

    # Get the current timestamp
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Write the results to a CSV file
    headers = ["time", "imgs", "text_prompt", "response", "matrix"]
    row = {
        "time": current_time,
        "imgs": str(setting["image_path"]),
        "text_prompt": str(text_prompt),
        "response": response.text,
        "matrix": str(matrix),
    }

    # Ensure the CSV file is created with headers if it doesn't exist
    try:
        with open(csv_file, mode="a", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            if file.tell() == 0:  # Write headers if the file is empty
                writer.writeheader()
            writer.writerow(row)
    except Exception as e:
        logger.error("Failed to write to CSV: %s", str(e))
        raise

    return response.text, matrix
  
def compose_content(dict_info):
  num_of_v = len(dict_info["variables"])
  variables = dict_info["variables"]
  content = ''
  for i,v in enumerate(variables):
    content += f"{i+1}. {variables[v]}\n"
  content = f"There are {num_of_v} variables: \n{content}.\n" 
  content += "Please fill this causality adjacency matrix:\n"
  return content

#### magnet

In [9]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info
from utils import evaluation

scene = info.scene()
scene_info_dict = scene.get_scene("Magnets")

magnet_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_magnetic_128P/"

files = os.listdir(magnet)
files = [magnet_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']
  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_]')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting)




NameError: name 'magnet' is not defined

In [ ]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("Magnets")

magnet_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_magnetic_128P/"

files = os.listdir(magnet)
files = [magnet_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']
  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_]')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = " Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_magnet_basic.csv")




In [350]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

In [106]:
import numpy as np
def cal_TPR_between_matrix(ground_truth_matrix1, matrix2):
    """
    Calculate the True Positive Rate between two adjacency matrix
    """
    # Check if the matrices are the same size
    if ground_truth_matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
      
    # Calculate the True Positive Rate
    TP = np.sum((ground_truth_matrix1 == 1) & (matrix2 == 1))
    FN = np.sum((ground_truth_matrix1 == 1) & (matrix2 == 0))
    TPR = TP / (TP + FN)
    
    return TPR
  
def cal_FPR_between_matrix(ground_truth_matrix1, matrix2):
    """
    Calculate the False Positive Rate between two adjacency matrix
    """
    # Check if the matrices are the same size
    if ground_truth_matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
      
    # Calculate the True Positive Rate
    FP = np.sum((ground_truth_matrix1 == 0) & (matrix2 == 1))
    TN = np.sum((ground_truth_matrix1 == 0) & (matrix2 == 0))
    FPR = FP / (FP + TN)
    
    return FPR
  
def cal_SHD_between_matrix(ground_truth_matrix1, matrix2):
    """
    Calculate the Structural Hamming Distance between two adjacency matrix
    """
    # Check if the matrices are the same size
    if ground_truth_matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
      
    # Calculate the Structural Hamming Distance
    SHD = np.sum(ground_truth_matrix1 != matrix2)
    
    return SHD
  
def cal_Accuarcy_between_matrix(ground_truth_matrix1, matrix2):
    """
    Calculate the Accuarcy between two adjacency matrix
    """
    # Check if the matrices are the same size
    if ground_truth_matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
      
    # Calculate the Structural Hamming Distance
    Accuarcy = np.sum(ground_truth_matrix1 == matrix2) / ground_truth_matrix1.size
    
    return Accuarcy
  
def cal_Precision_between_matrix(ground_truth_matrix1, matrix2):
    """
    Calculate the Precision between two adjacency matrix
    """
    # Check if the matrices are the same size
    if ground_truth_matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
      
    # Calculate the Structural Hamming Distance
    TP = np.sum((ground_truth_matrix1 == 1) & (matrix2 == 1))
    FP = np.sum((ground_truth_matrix1 == 0) & (matrix2 == 1))
    if (TP + FP) == 0:
      return 0
    Precision = TP / (TP + FP)
    
    return Precision
  
def cal_Recall_between_matrix(ground_truth_matrix1, matrix2):
    """
    Calculate the Recall between two adjacency matrix
    """
    # Check if the matrices are the same size
    if ground_truth_matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
      
    # Calculate the Structural Hamming Distance
    TP = np.sum((ground_truth_matrix1 == 1) & (matrix2 == 1))
    FN = np.sum((ground_truth_matrix1 == 1) & (matrix2 == 0))
    Recall = TP / (TP + FN)
    
    return Recall
  
def cal_F1_between_matrix(ground_truth_matrix1, matrix2):
    """
    Calculate the F1 between two adjacency matrix
    """
    # Check if the matrices are the same size
    if ground_truth_matrix1.shape != matrix2.shape:
        raise ValueError("Matrices must have the same shape")
      
    # Calculate the Structural Hamming Distance
    TP = np.sum((ground_truth_matrix1 == 1) & (matrix2 == 1))
    FP = np.sum((ground_truth_matrix1 == 0) & (matrix2 == 1))
    FN = np.sum((ground_truth_matrix1 == 1) & (matrix2 == 0))

    if (TP + FP) == 0:
      return 0
    Precision = TP / (TP + FP)    

    if (TP + FN) == 0:
      return 0
    Recall = TP / (TP + FN)
    F1 = 2 * Precision * Recall / (Precision + Recall)
    if (Precision + Recall) == 0:
      return 0
    
    return F1
  

In [349]:
all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 1], [0, 0, 1], [0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.0), np.float64(0.5)] 
TPR: 0.4864864864864865
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0

In [351]:
all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 1], [0, 0, 1], [0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.0), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5)] 
TPR: 0.4857142857142857
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0),

### Reflection

In [ ]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("reflection")

reflection_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_reflection_128P/"

files = os.listdir(reflection_img_path)
files = [reflection_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']
  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_reflection.csv")


In [372]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_reflection.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 1], [0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0)] 
TPR: 1.0
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
FPR: 0.0
[np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)] 
SHD: 0.0
[np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float

### spring


In [382]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("spring")

spring_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_spring_128P/"

files = os.listdir(spring_img_path)
files = [spring_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']
  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_spring.csv")


 80%|████████  | 8/10 [00:49<00:12,  6.23s/it]


ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff.

In [384]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_spring.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 1], [0, 0,1], [0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.5), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.5), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
TPR: 0.25
[np.float64(0.2857142857142857), np.float64(0.2857142857142857), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.42857142857142855), np.float64(0.0), np.float64(0.2857142857142857), np.float64(0.2857142857142857), np.float64(0.2857142857142857), np.float64(0.14285714285714285), np.float64(0.2857142857142857), np.float64(0.42857142857142855), np.float64(0.2857142857142857), np.float64(0.2857142857142857), np.float64(0.2857142857142857), np.float64(0.2857142857142857), np.float64(0.2857142857142857), np.float64(0.2857142857142857), np.float64(0.0), n

In [ ]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("spring")

spring_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_spring_128P/"

files = os.listdir(spring_img_path)
files = [spring_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']
  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_spring_basic.csv")


In [407]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_spring_basic.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 1], [0, 0,1], [0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.5), np.float64(1.0)] 
TPR: 0.7763157894736842
[np.float64(0.2857142857142857), np.float64(0.0), np.float64(0.2857142857142857), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)

### seesaw

In [ ]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("Seesaw")

seesaw_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_seesaw_128P/"

files = os.listdir(seesaw_img_path)
files = [seesaw_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_seesaw_basic.csv")


In [25]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_seesaw_basic.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5)] 
TPR: 0.5
[np.float64(0.0), np.float64(0.09523809523809523), np.float64(0.0), np.float64(0.09523809523809523), np.float64(0.0), np.float64(0.0), np.float64(0.09523809523809523), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
FPR: 0.02857142857142857
[np.int64(2), np.int64(4), np.int64(2), np.int64(4), np.int64(2), np.int64(2), np.int64(4), np.int64(2), np.int64(2), np.int64(2)] 
SHD: 2.6
[np.float64(1.0), np.float64(0.5), np.float64(1.0), np.float64(0.5), np.float64(1.0), np.float64(1.0), np.float64(0.5), np.float64(1.0), np.float64(1.0), np.float64(1.0)] 
Precision: 0.85
[np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5)] 
Recall: 0.5
[np.float64(0.92), np.float64(0.84), np.float64(0.9

In [27]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_seesaw.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.5), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.5), np.float64(0.5), np.float64(1.0), np.float64(1.0), np.float64(0.5), np.float64(0.5), np.float64(1.0), np.float64(1.0), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.5), np.float64(0.5), np.float64(1.0), np.float64(0.5), np.float64(0.5), np.float64(1.0), np.float64(0.5), np.float64(0.5), np.float64(1.0)] 
TPR: 0.7407407407407407
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.09523809523809523), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.09523809523809523), np.float64(0.0), np.float64(0.09523809523809523), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.09523809523809523), np.float64(0.09523809523809523), np.float64(0.0), np.float64(0.0),

### Refraction

In [ ]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("P_refraction")

transmission_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/transmission/"

files = os.listdir(transmission_img_path)
files = [transmission_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_transmission.csv")


In [52]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_transmission.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0)] 
TPR: 0.9595959595959597
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.07692307692307693), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.1538461

### prism reflection

In [ ]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("P_reflection")

prism_reflection_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/prism_reflection/"

files = os.listdir(prism_reflection_img_path)
files = [prism_reflection_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_P_reflection.csv")


In [57]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_P_reflection.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.6666666666666666), np.float64(1.0), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(1.0), np.float64(1.0), np.float64(0.6666666666666666), np.float64(1.0), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(1.0), np.float64(1.0), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(1.0), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(1.0), np.float64(1.0), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(0.0), np.float64(1.0)] 
TPR: 0.7261904761904762
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.15384615384615385), np.float64(0.0), np.float64(0.15384615384615385), np.float64(0.15384615384615385), np.float64(0.15384615384615385), np.float64(0.07692307692307693),

/tmp/ipykernel_2563904/2682095722.py:103: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


### hypythetical v3


In [ ]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("3_V")

h3_linear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h3_128P/"

files = os.listdir(h3_linear_img_path)
files = [h3_linear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h3_linear.csv")


In [79]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h3_linear.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 1, 1], [0, 0, 1], [0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(0.6666666666666666), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.6666666666666666), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333)] 
TPR: 0.37500000000000006

/tmp/ipykernel_2563904/3887620691.py:108: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


In [75]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h3_linear_basic.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 1, 1], [0, 0, 1], [0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.6666666666666666), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
TPR: 0.15151515151515152
[np.float64(0.0), np.float64(0.16666666666666666), np.float64(0.16666666666666666), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.16666666666666666), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.16666666666666666), np.float64(0.16666666666666666), np.float64(0.1666666666666

/tmp/ipykernel_2563904/3887620691.py:108: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


### hypothetic v4

In [83]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("4_V")

h4_linear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h4_100x256/"

files = os.listdir(h4_linear_img_path)
files = [h4_linear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h4_linear.csv")


100%|██████████| 10/10 [01:29<00:00,  8.99s/it]


In [85]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("4_V")

h4_linear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h4_100x256/"

files = os.listdir(h4_linear_img_path)
files = [h4_linear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h4_linear_base.csv")


100%|██████████| 10/10 [01:30<00:00,  9.09s/it]


In [86]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h4_linear_base.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0,1,1,0],[0,0,1,0],[0,0,0,0],[1,0,0,1]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.2), np.float64(0.2), np.float64(0.2), np.float64(0.2), np.float64(0.0), np.float64(0.2), np.float64(0.0), np.float64(0.2), np.float64(0.2), np.float64(0.2)] 
TPR: 0.15999999999999998
[np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.0), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091)] 
FPR: 0.08181818181818183
[np.int64(5), np.int64(5), np.int64(5), np.int64(4), np.int64(6), np.int64(5), np.int64(6), np.int64(5), np.int64(5), np.int64(5)] 
SHD: 5.1
[np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(1.0), np.float64(0.0), np.float64(0.5), np.float64(0.0), np.float64(0.5), np.float64(0.5), np.float64(0.5)] 
Precision: 0.45
[np.float64(0.2), np.float64(0.2), np.float64(0.2), np.float64(0.2), np.float64(0.0), np.float64(0.2), np.float64(0.0), np.

/tmp/ipykernel_2563904/3887620691.py:108: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


In [84]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h4_linear.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0,1,1,0],[0,0,1,0],[0,0,0,0],[1,0,0,1]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.2), np.float64(0.0), np.float64(0.2), np.float64(0.0), np.float64(0.6), np.float64(0.0), np.float64(0.4), np.float64(0.2), np.float64(0.6), np.float64(0.6), np.float64(0.6), np.float64(0.2), np.float64(0.2), np.float64(0.6), np.float64(0.2), np.float64(0.2), np.float64(0.0), np.float64(0.0), np.float64(0.6), np.float64(0.0), np.float64(0.2), np.float64(0.6), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.6), np.float64(0.2), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.4)] 
TPR: 0.23870967741935484
[np.float64(0.0), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.0), np.float64(0.0), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.0), np.float64(0.0), np.float64(0.09090909090909091), np.float64(0.09090909090909091), np.float64(0.0909090

/tmp/ipykernel_2563904/3887620691.py:108: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


### hyothetical V5

In [90]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("5_V")
h5_linear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h5_200x100/"

files = os.listdir(h5_linear_img_path)
files = [h5_linear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h5_linear.csv")


100%|██████████| 10/10 [01:54<00:00, 11.46s/it]


In [91]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h5_linear.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0,1,1,0,1],[0,0,1,0,0],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
TPR: 0.03571428571428571
[np.float64(0.16666666666666666), np.float64(0.2222222222222222), np.float64(0.1111111111111111), np.float64(0.1111111111111111), np.float64(0.1111111111111111), np.float64(0.16666666666666666), np.float64(0.0), np.float64(0.1111111111111111), np.float64(0.0), np.float64(0.2222222222222222), np.float64(0.16666666666666666), np.float64(0.2222222222222222), np.float64(0.1111111111111111), np.float64(0.16666666666666666), np.float64(0.1111111111111111), np.float64(0.16666666666666666), np.float64(0.1111111111111111), np.float64(0.1666666666

/tmp/ipykernel_2563904/3887620691.py:108: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


In [94]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info

scene = info.scene()
scene_info_dict = scene.get_scene("5_V")
h5_linear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h5_200x100/"

files = os.listdir(h5_linear_img_path)
files = [h5_linear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h5_linear_base.csv")


100%|██████████| 10/10 [01:58<00:00, 11.84s/it]


In [95]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h5_linear_base.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0,1,1,0,1],[0,0,1,0,0],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.14285714285714285), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.2857142857142857), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0)] 
TPR: 0.06285714285714285
[np.float64(0.05555555555555555), np.float64(0.05555555555555555), np.float64(0.16666666666666666), np.float64(0.05555555555555555), np.float64(0.2777777777777778), np.float64(0.05555555555555555), np.float64(0.05555555555555555), np.float64(0.05555555555555555), np.float64(0.1111111111111111), np.float64(0.1111111111111111), np.float64(0.05555555555555555), np.float64(0.27777

/tmp/ipykernel_2563904/3887620691.py:108: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


### hypothetic v2 -- Nonlinear

In [103]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info
import numpy as np

class scene:
  def __init__(self,):
    self.scenes = {
      "reflection": {
        #![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Reflection_circle_320x320.png)
        "variables": {0: "incident_degree", 1: "reflection_degree"},
        "adjacency_matrix": np.array([[0, 1], [0, 0]])
      },
      "spring": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Spring.png)
        "variables": {0: "spring_constant", 1: "weight", 2: "defomation"},
        "adjacency_matrix": np.array([[0,0,1],[0,0,1],[0,0,0]])
      },
      "Seesaw": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241120_150057.png)
        "variables": {0: "seesaw_left_arm", 1: "left_weight", 2: "seesaw_right_arm", 3: "right_weight", 4: "seesaw_torque"},
        "variables_2": {0: "left_force", 1: "right_force", 2: "seesaw_torque"},
        "adjacency_matrix_2": np.array([[0,0,1],[0,0,1],[0,0,0]]),
        "adjacency_matrix": np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1],
                                        [0, 0, 0, 0, 0]])
      },
      "Magnets": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241123_161201_Over_3D_256p.png)
        "variables": {0: "neddle_position", 1: "magnetic_bar_direction", 2: "neddle_direction"},
        "adjacency_matrix": np.array([[0, 0 , 1], [0, 0, 1], [0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/Yellow%20(570-590%20nm)_20241201_100615.png)
      "P_reflection": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "reflected_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/1732947800.585619_rendered_image.png)
      "P_refraction": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "refracted_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      "3_V": {
        "variables": {0: "v_ball", 1: "v_cylinder", 2: "angle"},
        "adjacency_matrix": np.array([[0, 1, 1], [0, 0, 1], [0, 0, 0]])
      },
      "4_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above"},
        "adjacency_matrix": np.array([[0,1,1,0],[0,0,1,0],[0,0,0,0],[1,0,0,1]])
      },
      "5_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above", 4: "angle"},
        "adjacency_matrix": np.array([[0,1,1,0,1],[0,0,1,0,0],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]])
      },
      "2_V_nonlinear": {
        "variables": {0: "volumn of cylinder", 1: "volumn of ball"},
        # b = cos(a) + 5 * epsilon
        "adjacency_matrix": np.array([[0,1],[0,0]])
      },
      
      
    }
    
  def get_all_scenes(self):
    return self.scenes
  
  def get_scencs_name(self):
    return self.scenes.keys()
  
  def get_scene(self, scene_name):
    return self.scenes[scene_name]

scene = scene()
scene_info_dict = scene.get_scene("2_V_nonlinear")
h2_nonlinear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h2_nonlinear_128P/"

files = os.listdir(h2_nonlinear_img_path)
files = [h2_nonlinear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h2_nonlinear_base.csv")


 70%|███████   | 7/10 [00:24<00:10,  3.53s/it]


ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff.

In [107]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h2_nonlinear.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0,1],[0,0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
TPR: 0.0
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
FPR: 0.0
[np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1)] 
SHD: 1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Precision: 0.0
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
Recall: 0.0
[np.float64(0.75), np.float64(0.75), np.float64(0.75), 

### hypothetic v3 Nonlinear 2

In [157]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info
import numpy as np

class scene:
  def __init__(self,):
    self.scenes = {
      "reflection": {
        #![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Reflection_circle_320x320.png)
        "variables": {0: "incident_degree", 1: "reflection_degree"},
        "adjacency_matrix": np.array([[0, 1], [0, 0]])
      },
      "spring": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Spring.png)
        "variables": {0: "spring_constant", 1: "weight", 2: "defomation"},
        "adjacency_matrix": np.array([[0,0,1],[0,0,1],[0,0,0]])
      },
      "Seesaw": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241120_150057.png)
        "variables": {0: "seesaw_left_arm", 1: "left_weight", 2: "seesaw_right_arm", 3: "right_weight", 4: "seesaw_torque"},
        "variables_2": {0: "left_force", 1: "right_force", 2: "seesaw_torque"},
        "adjacency_matrix_2": np.array([[0,0,1],[0,0,1],[0,0,0]]),
        "adjacency_matrix": np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1],
                                        [0, 0, 0, 0, 0]])
      },
      "Magnets": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241123_161201_Over_3D_256p.png)
        "variables": {0: "neddle_position", 1: "magnetic_bar_direction", 2: "neddle_direction"},
        "adjacency_matrix": np.array([[0, 0 , 1], [0, 0, 1], [0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/Yellow%20(570-590%20nm)_20241201_100615.png)
      "P_reflection": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "reflected_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/1732947800.585619_rendered_image.png)
      "P_refraction": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "refracted_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      "3_V": {
        "variables": {0: "v_ball", 1: "v_cylinder", 2: "angle"},
        "adjacency_matrix": np.array([[0, 1, 1], [0, 0, 1], [0, 0, 0]])
      },
      "4_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above"},
        "adjacency_matrix": np.array([[0,1,1,0],[0,0,1,0],[0,0,0,0],[1,0,0,1]])
      },
      "5_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above", 4: "angle"},
        "adjacency_matrix": np.array([[0,1,1,0,1],[0,0,1,0,0],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]])
      },
      "2_V_nonlinear": {
        "variables": {0: "volumn of cylinder", 1: "volumn of ball"},
        # b = cos(a) + 5 * epsilon
        "adjacency_matrix": np.array([[0,1],[0,0]])
      },
      "3_V_nonlinear_1": {
        "variables": {0: "volumn of ball", 1: "volumn of cylinder", 2: "tile angle of the rectangular prism"},
        "adjacency_matrix": np.array([[0, 0, 1], [1, 0, 1], [0, 0, 0]])
      },
      
      "3_V_nonlinear_2": {
        "variables": {0: "volumn of ball", 1: "volumn of cylinder", 2: "tile angle of the rectangular prism"},
        "adjacency_matrix": np.array([[0, 0, 1], [0, 0, 1], [0, 0, 0]])
      },
      
    }
    
  def get_all_scenes(self):
    return self.scenes
  
  def get_scencs_name(self):
    return self.scenes.keys()
  
  def get_scene(self, scene_name):
    return self.scenes[scene_name]

scene = scene()
scene_info_dict = scene.get_scene("3_V_nonlinear_2")
h3_nonlinear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h3_nonlinear_1_128P/"

files = os.listdir(h3_nonlinear_img_path)
files = [h3_nonlinear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h3_nonlinear_2_base.csv")


100%|██████████| 10/10 [00:59<00:00,  5.91s/it]


In [158]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h3_nonlinear_2_base.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 1], [0, 0, 1], [0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.5), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.5), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.0), np.float64(0.5), np.float64(0.5), np.float64(0.0), np.float64(0.5), np.float64(0.0), np.float64(0.0), np.float64(0.5), np.float64(0.5), np.float64(0.0), np.float64(0.0), np.float64(0.5), np.float64(0.5)] 
TPR: 0.20689655172413793
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.14285714285714285), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.

In [161]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info
import numpy as np

class scene:
  def __init__(self,):
    self.scenes = {
      "reflection": {
        #![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Reflection_circle_320x320.png)
        "variables": {0: "incident_degree", 1: "reflection_degree"},
        "adjacency_matrix": np.array([[0, 1], [0, 0]])
      },
      "spring": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Spring.png)
        "variables": {0: "spring_constant", 1: "weight", 2: "defomation"},
        "adjacency_matrix": np.array([[0,0,1],[0,0,1],[0,0,0]])
      },
      "Seesaw": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241120_150057.png)
        "variables": {0: "seesaw_left_arm", 1: "left_weight", 2: "seesaw_right_arm", 3: "right_weight", 4: "seesaw_torque"},
        "variables_2": {0: "left_force", 1: "right_force", 2: "seesaw_torque"},
        "adjacency_matrix_2": np.array([[0,0,1],[0,0,1],[0,0,0]]),
        "adjacency_matrix": np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1],
                                        [0, 0, 0, 0, 0]])
      },
      "Magnets": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241123_161201_Over_3D_256p.png)
        "variables": {0: "neddle_position", 1: "magnetic_bar_direction", 2: "neddle_direction"},
        "adjacency_matrix": np.array([[0, 0 , 1], [0, 0, 1], [0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/Yellow%20(570-590%20nm)_20241201_100615.png)
      "P_reflection": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "reflected_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/1732947800.585619_rendered_image.png)
      "P_refraction": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "refracted_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      "3_V": {
        "variables": {0: "v_ball", 1: "v_cylinder", 2: "angle"},
        "adjacency_matrix": np.array([[0, 1, 1], [0, 0, 1], [0, 0, 0]])
      },
      "4_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above"},
        "adjacency_matrix": np.array([[0,1,1,0],[0,0,1,0],[0,0,0,0],[1,0,0,1]])
      },
      "5_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above", 4: "angle"},
        "adjacency_matrix": np.array([[0,1,1,0,1],[0,0,1,0,0],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]])
      },
      "2_V_nonlinear": {
        "variables": {0: "volumn of cylinder", 1: "volumn of ball"},
        # b = cos(a) + 5 * epsilon
        "adjacency_matrix": np.array([[0,1],[0,0]])
      },
      "3_V_nonlinear_1": {
        "variables": {0: "volumn of ball", 1: "volumn of cylinder", 2: "tile angle of the rectangular prism"},
        "adjacency_matrix": np.array([[0, 0, 1], [1, 0, 1], [0, 0, 0]])
      },
      
      "3_V_nonlinear_2": {
        "variables": {0: "volumn of ball", 1: "volumn of cylinder", 2: "tile angle of the rectangular prism"},
        "adjacency_matrix": np.array([[0, 0, 1], [0, 0, 1], [0, 0, 0]])
      },
      
    }
    
  def get_all_scenes(self):
    return self.scenes
  
  def get_scencs_name(self):
    return self.scenes.keys()
  
  def get_scene(self, scene_name):
    return self.scenes[scene_name]

scene = scene()
scene_info_dict = scene.get_scene("3_V_nonlinear_2")
h3_nonlinear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h3_nonlinear_1_128P/"

files = os.listdir(h3_nonlinear_img_path)
files = [h3_nonlinear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h3_nonlinear_2.csv")


  0%|          | 0/10 [00:00<?, ?it/s]


ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff.

In [162]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h3_nonlinear_2.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0, 0, 1], [1, 0, 1], [0, 0, 0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.0)] 
TPR: 0.1212121212121212
[np.float64(0.0), np.float64(0.16666666666666666), np.float64(0.16666666666666666), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.16666666666666666)] 
FPR: 0.045454545454545456
[np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(2), np.int64(3), np.int64(3), np.int64(3), np.int64(2), np.int64(4)] 
SHD: 2.909090909090909
[0, np.float64(0.5), np.float64(0.5), 0, 0, np.float64(1.0), 0, 0, 0, np.float64(1.0), np.float64(0.0)] 
Precision: 0.2727272727272727
[np.float64(0.0), np.float64(0.3333333333333333), np.float64(0.3333333333333333), np.float64(0.0), np.float64(0.0), np.float64(0.3333333333333333), np.

/tmp/ipykernel_2563904/133219692.py:110: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


### hpyothetic v_4 nonlinear

In [144]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info
import numpy as np

class scene:
  def __init__(self,):
    self.scenes = {
      "reflection": {
        #![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Reflection_circle_320x320.png)
        "variables": {0: "incident_degree", 1: "reflection_degree"},
        "adjacency_matrix": np.array([[0, 1], [0, 0]])
      },
      "spring": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Spring.png)
        "variables": {0: "spring_constant", 1: "weight", 2: "defomation"},
        "adjacency_matrix": np.array([[0,0,1],[0,0,1],[0,0,0]])
      },
      "Seesaw": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241120_150057.png)
        "variables": {0: "seesaw_left_arm", 1: "left_weight", 2: "seesaw_right_arm", 3: "right_weight", 4: "seesaw_torque"},
        "variables_2": {0: "left_force", 1: "right_force", 2: "seesaw_torque"},
        "adjacency_matrix_2": np.array([[0,0,1],[0,0,1],[0,0,0]]),
        "adjacency_matrix": np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1],
                                        [0, 0, 0, 0, 0]])
      },
      "Magnets": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241123_161201_Over_3D_256p.png)
        "variables": {0: "neddle_position", 1: "magnetic_bar_direction", 2: "neddle_direction"},
        "adjacency_matrix": np.array([[0, 0 , 1], [0, 0, 1], [0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/Yellow%20(570-590%20nm)_20241201_100615.png)
      "P_reflection": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "reflected_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/1732947800.585619_rendered_image.png)
      "P_refraction": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "refracted_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      "3_V": {
        "variables": {0: "v_ball", 1: "v_cylinder", 2: "angle"},
        "adjacency_matrix": np.array([[0, 1, 1], [0, 0, 1], [0, 0, 0]])
      },
      "4_V_nonlinear": {
        "variables": {0: "volumn of ball", 1: "volumn of cylinder", 2: "distance from ball to cylinder",
                      3: "cylinder's height above the ground"},
        "adjacency_matrix": np.array([[0,0,1,0],[0,0,1,0],[0,0,0,1],[0,0,0,0]])
      },
      "5_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above", 4: "angle"},
        "adjacency_matrix": np.array([[0,1,1,0,1],[0,0,1,0,0],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]])
      },
      "5_V_nonlinear": {
        "variables": {0: "volumn of ball", 1: "height of cylinder", 2: "distance between ball and cylinder", 
                      3: "cylinder's height above the ground", 4: "tilt angle of cylinder"},
        "adjacency_matrix": np.array([[0,0,1,0,0],[0,0,1,0,0],[0,0,0,0,1],[0,1,0,0,1],[0,0,0,0,0]])
      },
      "2_V_nonlinear": {
        "variables": {0: "volumn of cylinder", 1: "volumn of ball"},
        # b = cos(a) + 5 * epsilon
        "adjacency_matrix": np.array([[0,1],[0,0]])
      },
      "3_V_nonlinear_1": {
        "variables": {0: "volumn of ball", 1: "volumn of cylinder", 2: "tile angle of the rectangular prism"},
        "adjacency_matrix": np.array([[0, 0, 1], [1, 0, 1], [0, 0, 0]])
      },
      
      
    }
    
  def get_all_scenes(self):
    return self.scenes
  
  def get_scencs_name(self):
    return self.scenes.keys()
  
  def get_scene(self, scene_name):
    return self.scenes[scene_name]

scene = scene()
scene_info_dict = scene.get_scene("4_V_nonlinear")
h4_nonlinear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h4_128x128_nonlinear/"

files = os.listdir(h4_nonlinear_img_path)
files = [h4_nonlinear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h4_nonlinear.csv")


  0%|          | 0/10 [00:08<?, ?it/s]


SyntaxError: invalid syntax (<string>, line 2)

In [148]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h4_nonlinear_base.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0,0,1,0],[0,0,1,0],[0,0,0,1],[0,0,0,0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)] 
TPR: 0.0
[np.float64(0.07692307692307693), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.07692307692307693), np.float64(0.07692307692307693), np.float64(0.0), np.float64(0.07692307692307693), np.float64(0.07692307692307693), np.float64(0.07692307692307693), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.07692307692307693), np.float64(0.07692307692307693)] 
FPR: 0.03619909502262444
[np.int64(4), np.int64(3), np.int64(3), np.int64(3), np.int64(4), np.int64(4), np.int64(3), np.int64(4), np.int64(4), np.int64(4), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(4), np.int64(4)] 
SHD: 3.4705882352941178

/tmp/ipykernel_2563904/133219692.py:110: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


### hpyothetic v_5 nonlinear

In [129]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info
import numpy as np

class scene:
  def __init__(self,):
    self.scenes = {
      "reflection": {
        #![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Reflection_circle_320x320.png)
        "variables": {0: "incident_degree", 1: "reflection_degree"},
        "adjacency_matrix": np.array([[0, 1], [0, 0]])
      },
      "spring": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Spring.png)
        "variables": {0: "spring_constant", 1: "weight", 2: "defomation"},
        "adjacency_matrix": np.array([[0,0,1],[0,0,1],[0,0,0]])
      },
      "Seesaw": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241120_150057.png)
        "variables": {0: "seesaw_left_arm", 1: "left_weight", 2: "seesaw_right_arm", 3: "right_weight", 4: "seesaw_torque"},
        "variables_2": {0: "left_force", 1: "right_force", 2: "seesaw_torque"},
        "adjacency_matrix_2": np.array([[0,0,1],[0,0,1],[0,0,0]]),
        "adjacency_matrix": np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1],
                                        [0, 0, 0, 0, 0]])
      },
      "Magnets": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241123_161201_Over_3D_256p.png)
        "variables": {0: "neddle_position", 1: "magnetic_bar_direction", 2: "neddle_direction"},
        "adjacency_matrix": np.array([[0, 0 , 1], [0, 0, 1], [0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/Yellow%20(570-590%20nm)_20241201_100615.png)
      "P_reflection": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "reflected_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/1732947800.585619_rendered_image.png)
      "P_refraction": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "refracted_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      "3_V": {
        "variables": {0: "v_ball", 1: "v_cylinder", 2: "angle"},
        "adjacency_matrix": np.array([[0, 1, 1], [0, 0, 1], [0, 0, 0]])
      },
      "4_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above"},
        "adjacency_matrix": np.array([[0,1,1,0],[0,0,1,0],[0,0,0,0],[1,0,0,1]])
      },
      "5_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above", 4: "angle"},
        "adjacency_matrix": np.array([[0,1,1,0,1],[0,0,1,0,0],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]])
      },
      "5_V_nonlinear": {
        "variables": {0: "volumn of ball", 1: "height of cylinder", 2: "distance between ball and cylinder", 
                      3: "cylinder's height above the ground", 4: "tilt angle of cylinder"},
        "adjacency_matrix": np.array([[0,0,1,0,0],[0,0,1,0,0],[0,0,0,0,1],[0,1,0,0,1],[0,0,0,0,0]])
      },
      "2_V_nonlinear": {
        "variables": {0: "volumn of cylinder", 1: "volumn of ball"},
        # b = cos(a) + 5 * epsilon
        "adjacency_matrix": np.array([[0,1],[0,0]])
      },
      "3_V_nonlinear_1": {
        "variables": {0: "volumn of ball", 1: "volumn of cylinder", 2: "tile angle of the rectangular prism"},
        "adjacency_matrix": np.array([[0, 0, 1], [1, 0, 1], [0, 0, 0]])
      },
      
      
    }
    
  def get_all_scenes(self):
    return self.scenes
  
  def get_scencs_name(self):
    return self.scenes.keys()
  
  def get_scene(self, scene_name):
    return self.scenes[scene_name]

scene = scene()
scene_info_dict = scene.get_scene("5_V_nonlinear")
h5_nonlinear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h5_150x150_nonlinear/"

files = os.listdir(h5_nonlinear_img_path)
files = [h5_nonlinear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "Please analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h5_nonlinear_base.csv")


 70%|███████   | 7/10 [01:02<00:26,  9.00s/it]


ResourceExhausted: 429 Online prediction request quota exceeded for gemini-1.5-pro. Please try again later with backoff.

In [131]:
import sys
import os
import random
from tqdm import tqdm
sys.path.append('/home/lds/github/Causality-informed-Generation/inference/evaluation')
from utils import info
import numpy as np

class scene:
  def __init__(self,):
    self.scenes = {
      "reflection": {
        #![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Reflection_circle_320x320.png)
        "variables": {0: "incident_degree", 1: "reflection_degree"},
        "adjacency_matrix": np.array([[0, 1], [0, 0]])
      },
      "spring": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/render_output_Blank_Spring.png)
        "variables": {0: "spring_constant", 1: "weight", 2: "defomation"},
        "adjacency_matrix": np.array([[0,0,1],[0,0,1],[0,0,0]])
      },
      "Seesaw": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241120_150057.png)
        "variables": {0: "seesaw_left_arm", 1: "left_weight", 2: "seesaw_right_arm", 3: "right_weight", 4: "seesaw_torque"},
        "variables_2": {0: "left_force", 1: "right_force", 2: "seesaw_torque"},
        "adjacency_matrix_2": np.array([[0,0,1],[0,0,1],[0,0,0]]),
        "adjacency_matrix": np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1],
                                        [0, 0, 0, 0, 0]])
      },
      "Magnets": {
        # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/20241123_161201_Over_3D_256p.png)
        "variables": {0: "neddle_position", 1: "magnetic_bar_direction", 2: "neddle_direction"},
        "adjacency_matrix": np.array([[0, 0 , 1], [0, 0, 1], [0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/Yellow%20(570-590%20nm)_20241201_100615.png)
      "P_reflection": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "reflected_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      # ![](https://cdn.jsdelivr.net/gh/DishengL/ResearchPics/1732947800.585619_rendered_image.png)
      "P_refraction": {
        "variables": {0: "wave_length", 1: "incident_position", 2: "incident_angle", 3: "refracted_position"},
        "adjacency_matrix": np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]])
      },
      "3_V": {
        "variables": {0: "v_ball", 1: "v_cylinder", 2: "angle"},
        "adjacency_matrix": np.array([[0, 1, 1], [0, 0, 1], [0, 0, 0]])
      },
      "4_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above"},
        "adjacency_matrix": np.array([[0,1,1,0],[0,0,1,0],[0,0,0,0],[1,0,0,1]])
      },
      "5_V": {
        "variables": {0: "v_ball", 1: "h_cylinder", 2: "d_ball_cylinder", 3: "cylinder_h_above", 4: "angle"},
        "adjacency_matrix": np.array([[0,1,1,0,1],[0,0,1,0,0],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]])
      },
      "5_V_nonlinear": {
        "variables": {0: "volumn of ball", 1: "height of cylinder", 2: "distance between ball and cylinder", 
                      3: "cylinder's height above the ground", 4: "tilt angle of cylinder"},
        "adjacency_matrix": np.array([[0,0,1,0,0],[0,0,1,0,0],[0,0,0,0,1],[0,1,0,0,1],[0,0,0,0,0]])
      },
      "2_V_nonlinear": {
        "variables": {0: "volumn of cylinder", 1: "volumn of ball"},
        # b = cos(a) + 5 * epsilon
        "adjacency_matrix": np.array([[0,1],[0,0]])
      },
      "3_V_nonlinear_1": {
        "variables": {0: "volumn of ball", 1: "volumn of cylinder", 2: "tile angle of the rectangular prism"},
        "adjacency_matrix": np.array([[0, 0, 1], [1, 0, 1], [0, 0, 0]])
      },
      
      
    }
    
  def get_all_scenes(self):
    return self.scenes
  
  def get_scencs_name(self):
    return self.scenes.keys()
  
  def get_scene(self, scene_name):
    return self.scenes[scene_name]

scene = scene()
scene_info_dict = scene.get_scene("5_V_nonlinear")
h5_nonlinear_img_path = "/home/lds/github/Causality-informed-Generation/code1/database/rendered_h5_150x150_nonlinear/"

files = os.listdir(h5_nonlinear_img_path)
files = [h5_nonlinear_img_path + file for file in files]
# randomly pick 10 images
for i in tqdm(range(10)):
  imgs_path = random.sample(files, 10)
  matrix = scene_info_dict['adjacency_matrix']

  matrix = str(matrix).replace("1", "_,").replace("0", "_,").replace("_,]", '_],')
  scene_info = compose_content(scene_info_dict)
  matrix_info = ".\nIn the matrix, matrix[i][j] = 1 means variable i causes variable j, matrix[i][j] = 0 means there is not direct causal relationship."
  system_info = "You are a causal discovery expert. Your task is to analyze the given images and identify any causal relationships present. Provide a brief explanation of the discovered causal relationships to support your conclusions."
  
  setting = {
  "image_path" : imgs_path,
  "system_info" : system_info,
  "scene_info" : scene_info,
  "matrix" : matrix,
  "matrix_info" : matrix_info,
  "model_name" : "gemini-1.5-pro-002",
  
}

  text, matrix = Gemini_infer(setting, csv_file="gemini_inference_log_h5_nonlinear.csv")


100%|██████████| 10/10 [01:56<00:00, 11.65s/it]


In [134]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h5_nonlinear_base.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0,0,1,0,0],[0,0,1,0,0],[0,0,0,0,1],[0,1,0,0,1],[0,0,0,0,0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.4), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.2), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.2), np.float64(0.0)] 
TPR: 0.042105263157894736
[np.float64(0.0), np.float64(0.0), np.float64(0.1), np.float64(0.05), np.float64(0.05), np.float64(0.05), np.float64(0.1), np.float64(0.0), np.float64(0.1), np.float64(0.1), np.float64(0.05), np.float64(0.0), np.float64(0.1), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.05), np.float64(0.0), np.float64(0.0)] 
FPR: 0.039473684210526314
[np.int64(5), np.int64(5), np.int64(7), np.int64(6), np.int64(6), np.int64(4), np.int64(7), np.int64(5), np.int64(7), np.int64(7), np.int64(6), np.int64(4), np.int64(7), np.int64(5), np.int64(5), np.int64(5), np.int64(6), np.int64(4), np.int64(5)] 
SHD: 5.578947368421052
[0, 0, np.f

/tmp/ipykernel_2563904/133219692.py:110: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)


In [133]:
csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_h5_nonlinear.csv"
# csv_file = "/home/lds/github/Causality-informed-Generation/inference/google_cloud/gemini_inference_log_magnet_basic.csv"
import pandas as pd
df = pd.read_csv(csv_file)
(list(df['matrix']))
inference = (list(df['matrix']))
inference = [eval(i) for i in inference]

all_tprs = []
all_fprs = []
all_shds = []
all_precisions = []
all_recalls = []
all_accuracys = []
all_F1s = []

ground_truth = np.array([[0,0,1,0,0],[0,0,1,0,0],[0,0,0,0,1],[0,1,0,0,1],[0,0,0,0,0]])
for matrix in inference:
    matrix = np.array(matrix)
    tpr = cal_TPR_between_matrix(ground_truth, matrix)
    fpr = cal_FPR_between_matrix(ground_truth, matrix)
    shd = cal_SHD_between_matrix(ground_truth, matrix)
    precision = cal_Precision_between_matrix(ground_truth, matrix)
    recall = cal_Recall_between_matrix(ground_truth, matrix)
    accuracy = cal_Accuarcy_between_matrix(ground_truth, matrix)
    F1 = cal_F1_between_matrix(ground_truth, matrix)
    
    all_tprs.append(tpr)
    all_fprs.append(fpr)
    
    all_shds.append(shd)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_accuracys.append(accuracy)
    all_F1s.append(F1)
print("explicted")
print(all_tprs, '\nTPR:', sum(all_tprs)/len(all_tprs))
print(all_fprs, '\nFPR:', sum(all_fprs)/len(all_fprs))
print(all_shds, '\nSHD:', sum(all_shds)/len(all_shds))
print(all_precisions, '\nPrecision:', sum(all_precisions)/len(all_precisions))
print(all_recalls, '\nRecall:', sum(all_recalls)/len(all_recalls))
print(all_accuracys, '\nAccuracy:', sum(all_accuracys)/len(all_accuracys))
print(all_F1s, '\nF1:', sum(all_F1s)/len(all_F1s))

explicted
[np.float64(0.2), np.float64(0.2), np.float64(0.4), np.float64(0.0), np.float64(0.2), np.float64(0.0), np.float64(0.2), np.float64(0.0), np.float64(0.0), np.float64(0.2)] 
TPR: 0.13999999999999999
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.05), np.float64(0.0), np.float64(0.05), np.float64(0.0), np.float64(0.1), np.float64(0.0), np.float64(0.0)] 
FPR: 0.02
[np.int64(4), np.int64(4), np.int64(3), np.int64(6), np.int64(4), np.int64(6), np.int64(4), np.int64(7), np.int64(5), np.int64(4)] 
SHD: 4.7
[np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), 0, np.float64(1.0)] 
Precision: 0.6
[np.float64(0.2), np.float64(0.2), np.float64(0.4), np.float64(0.0), np.float64(0.2), np.float64(0.0), np.float64(0.2), np.float64(0.0), np.float64(0.0), np.float64(0.2)] 
Recall: 0.13999999999999999
[np.float64(0.84), np.float64(0.84), np.float64(0.88), np.float64(0.76), np.float64(0.84), np.f

/tmp/ipykernel_2563904/133219692.py:110: RuntimeWarning: invalid value encountered in scalar divide
  F1 = 2 * Precision * Recall / (Precision + Recall)
